# micro-rnn: Understanding RNNs from Scratch

This notebook walks through the implementation of a vanilla RNN for character-level language modeling.

**What you'll learn:**
1. How RNNs process sequential data
2. Forward pass through time
3. Backpropagation Through Time (BPTT)
4. The vanishing gradient problem
5. Why LSTMs/GRUs were invented

**Prerequisites:**
- Karpathy's micrograd (backprop, neural networks)
- Basic Python and NumPy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)  # For reproducibility

## 1. The Problem: Sequential Data

Regular neural networks (MLPs) take fixed-size inputs. But what about:
- Text (variable length sentences)
- Audio (time series)
- Video (sequence of frames)

We need a network that can process **sequences** and maintain **memory** of what it has seen.

Enter the **Recurrent Neural Network (RNN)**.

## 2. RNN Architecture

The key idea: **hidden state** that gets updated at each timestep.

```
h_t = tanh(W_xh @ x_t + W_hh @ h_{t-1} + b_h)
y_t = W_hy @ h_t + b_y
```

Where:
- `x_t` = input at time t
- `h_t` = hidden state at time t (the "memory")
- `y_t` = output at time t
- `W_hh` = **recurrent weight** (this is what makes it an RNN!)

The hidden state `h_t` carries information from all previous timesteps.

## 3. Let's Build It!

First, let's implement the core RNN cell:

In [ ]:
class SimpleRNN:
    """
    Minimal RNN for character-level language modeling.
    """
    
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights with small random values
        self.W_xh = np.random.randn(hidden_size, input_size) * 0.01   # input -> hidden
        self.W_hh = np.random.randn(hidden_size, hidden_size) * 0.01  # hidden -> hidden (RECURRENT!)
        self.W_hy = np.random.randn(output_size, hidden_size) * 0.01  # hidden -> output
        
        # Biases
        self.b_h = np.zeros((hidden_size, 1))
        self.b_y = np.zeros((output_size, 1))
        
        self.hidden_size = hidden_size
    
    def forward_step(self, x, h_prev):
        """
        Single timestep forward pass.
        
        This is the heart of the RNN - it combines:
        - Current input (x)
        - Previous memory (h_prev)
        To produce new memory (h) and output (y)
        """
        # New hidden state = tanh(input contribution + memory contribution)
        h = np.tanh(
            np.dot(self.W_xh, x) +      # What the current input says
            np.dot(self.W_hh, h_prev) +  # What we remember from before
            self.b_h
        )
        
        # Output logits
        y = np.dot(self.W_hy, h) + self.b_y
        
        # Softmax for probabilities
        p = np.exp(y - np.max(y))
        p = p / np.sum(p)
        
        return h, y, p

# Create a small RNN
vocab_size = 5
hidden_size = 3
rnn = SimpleRNN(vocab_size, hidden_size, vocab_size)

print(f"Created RNN with:")
print(f"  - Input size: {vocab_size}")
print(f"  - Hidden size: {hidden_size}")
print(f"  - Output size: {vocab_size}")

## 4. Forward Pass Through Time

Let's see how the hidden state evolves as we feed in a sequence:

In [ ]:
# Create a sequence of 4 one-hot encoded inputs
sequence = [0, 2, 1, 3]  # character indices

# Initialize hidden state to zeros
h = np.zeros((hidden_size, 1))
print(f"Initial hidden state: {h.flatten()}")
print("\nProcessing sequence...\n")

hidden_states = [h.flatten()]

for t, idx in enumerate(sequence):
    # One-hot encode input
    x = np.zeros((vocab_size, 1))
    x[idx] = 1
    
    # Forward step
    h, y, p = rnn.forward_step(x, h)
    hidden_states.append(h.flatten())
    
    print(f"t={t}: input={idx}, hidden={h.flatten().round(3)}, top_pred={np.argmax(p)}")

# Visualize hidden state evolution
hidden_states = np.array(hidden_states)

plt.figure(figsize=(10, 4))
for i in range(hidden_size):
    plt.plot(hidden_states[:, i], marker='o', label=f'h[{i}]')
plt.xlabel('Timestep')
plt.ylabel('Activation')
plt.title('Hidden State Evolution Over Time')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\nNotice how each hidden unit changes as it processes the sequence!")
print("This is the RNN's 'memory' being updated.")

## 5. Character-Level Language Modeling

Our task: given a sequence of characters, predict the next character.

```
Input:  h e l l
Target: e l l o
```

Let's set up a real dataset:

In [ ]:
# Our training data
text = "hello world hello rnn hello deep learning"

# Create vocabulary
chars = sorted(list(set(text)))
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)

print(f"Text: '{text}'")
print(f"Vocabulary size: {vocab_size}")
print(f"Characters: {chars}")

# Convert text to indices
data = [char_to_idx[ch] for ch in text]
print(f"\nAs indices: {data[:20]}...")

## 6. The Backward Pass: BPTT

Now the tricky part: **Backpropagation Through Time (BPTT)**.

The key insight: gradients flow backward through the `W_hh` connections.

At each timestep, gradient gets multiplied by `W_hh`. If we have T timesteps:
- Gradient ~ `(W_hh)^T`

This causes problems:
- If eigenvalues of W_hh < 1: gradient **vanishes** (shrinks exponentially)
- If eigenvalues of W_hh > 1: gradient **explodes** (grows exponentially)

In [ ]:
# Let's see the vanishing gradient in action!
from rnn import RNN, create_dataset, get_batch

# Create RNN
char_to_idx, idx_to_char, data = create_dataset(text)
vocab_size = len(char_to_idx)
hidden_size = 50
rnn = RNN(vocab_size, hidden_size, vocab_size)

# Test with different sequence lengths
seq_lengths = [5, 10, 20, 30]

print("Gradient magnitude at first timestep vs last timestep:\n")

for seq_len in seq_lengths:
    # Create dummy data
    inputs = list(np.random.randint(0, vocab_size, seq_len))
    targets = list(np.random.randint(0, vocab_size, seq_len))
    
    h_prev = np.zeros((hidden_size, 1))
    loss, cache, _ = rnn.forward(inputs, targets, h_prev)
    
    xs, hs, ps = cache
    
    # Track gradient at each timestep
    dh_next = np.zeros((hidden_size, 1))
    grad_mags = []
    
    for t in reversed(range(seq_len)):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dh = np.dot(rnn.W_hy.T, dy) + dh_next
        dh_raw = (1 - hs[t] ** 2) * dh
        grad_mags.append(np.linalg.norm(dh_raw))
        dh_next = np.dot(rnn.W_hh.T, dh_raw)
    
    grad_mags = grad_mags[::-1]
    ratio = grad_mags[-1] / (grad_mags[0] + 1e-10)
    
    print(f"Seq length {seq_len:2d}: first={grad_mags[0]:.4f}, last={grad_mags[-1]:.4f}, ratio={ratio:.2f}x")

## 7. Visualizing the Vanishing Gradient

Let's plot how gradient magnitude changes across timesteps:

In [ ]:
# Longer sequence to see the effect clearly
seq_len = 50
inputs = list(np.random.randint(0, vocab_size, seq_len))
targets = list(np.random.randint(0, vocab_size, seq_len))

h_prev = np.zeros((hidden_size, 1))
loss, cache, _ = rnn.forward(inputs, targets, h_prev)
xs, hs, ps = cache

# Backward pass tracking gradients
dh_next = np.zeros((hidden_size, 1))
grad_mags = []

for t in reversed(range(seq_len)):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1
    dh = np.dot(rnn.W_hy.T, dy) + dh_next
    dh_raw = (1 - hs[t] ** 2) * dh
    grad_mags.append(np.linalg.norm(dh_raw))
    dh_next = np.dot(rnn.W_hh.T, dh_raw)

grad_mags = grad_mags[::-1]

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(grad_mags, 'b-', linewidth=2)
plt.xlabel('Timestep')
plt.ylabel('Gradient Magnitude')
plt.title('Gradient Flow Through Time')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.semilogy(grad_mags, 'r-', linewidth=2)
plt.xlabel('Timestep')
plt.ylabel('Gradient Magnitude (log scale)')
plt.title('Gradient Flow (Log Scale)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey insight: Gradients for early timesteps are MUCH smaller!")
print("This means the RNN struggles to learn long-range dependencies.")
print("\nSolution: LSTM and GRU use 'gates' to control gradient flow.")

## 8. Training the RNN

Now let's train our RNN to generate text:

In [ ]:
# Training parameters
hidden_size = 64
seq_length = 15
learning_rate = 0.1
num_iterations = 3000

# Fresh RNN
rnn = RNN(vocab_size, hidden_size, vocab_size)

# Training loop
losses = []
smooth_loss = -np.log(1.0 / vocab_size) * seq_length

n = 0
p = 0
h_prev = np.zeros((hidden_size, 1))

print("Training...\n")

for n in range(num_iterations):
    # Reset if at end of data
    if p + seq_length + 1 >= len(data):
        p = 0
        h_prev = np.zeros((hidden_size, 1))
    
    # Get batch
    inputs, targets = get_batch(data, p, seq_length)
    
    # Forward
    loss, cache, h_prev = rnn.forward(inputs, targets, h_prev)
    
    # Backward
    grads = rnn.backward(inputs, targets, cache)
    grads = rnn.clip_gradients(grads)
    
    # Update
    rnn.update_parameters(grads, learning_rate)
    
    # Track loss
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    losses.append(smooth_loss)
    
    if n % 500 == 0:
        print(f"Iteration {n}: loss = {smooth_loss:.4f}")
    
    p += seq_length

print(f"\nFinal loss: {smooth_loss:.4f}")

# Plot loss
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

## 9. Generate Text!

Let's see what our trained RNN can produce:

In [ ]:
# Generate text
def generate_text(rnn, seed_char, length):
    h = np.zeros((rnn.hidden_size, 1))
    seed_idx = char_to_idx[seed_char]
    indices = rnn.sample(seed_idx, h, length)
    return seed_char + ''.join(idx_to_char[i] for i in indices)

print("Generated text samples:\n")

for seed in ['h', 'w', 'd']:
    generated = generate_text(rnn, seed, 50)
    print(f"Seed '{seed}': {generated}")
    print()

## 10. Key Takeaways

### What we learned:

1. **RNN Architecture**: Hidden state carries memory through time via W_hh

2. **Forward Pass**: h_t = tanh(W_xh @ x_t + W_hh @ h_{t-1} + b_h)

3. **BPTT**: Gradients flow backward through W_hh connections

4. **Vanishing Gradient**: Gradients shrink exponentially for long sequences

5. **Gradient Clipping**: Simple fix for exploding gradients

### Why this matters:

The vanishing gradient problem is why **LSTMs and GRUs** were invented!

They use **gates** to control gradient flow:
- Forget gate: what to forget from memory
- Input gate: what new info to add
- Output gate: what to output

This allows gradients to flow unchanged through many timesteps.

### Next steps:

1. **micro-lstm**: Implement LSTM from scratch to see how gates work
2. **micro-gru**: Simplified version with fewer gates
3. **micro-seq2seq**: Encoder-decoder architecture for translation

Then you'll be ready for **attention** and **transformers**!

In [ ]:
print("Congratulations! You now understand RNNs from scratch!")
print("\nKey formula to remember:")
print("  h_t = tanh(W_xh @ x_t + W_hh @ h_{t-1} + b_h)")
print("\nThe W_hh is what makes it 'recurrent' - and causes vanishing gradients!")